# **Performance evaluation of the HMMs obtained**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, auc
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

## Functions

In [ ]:
# function that takes as input an array containing E-values and a threshold and produces predictions based on them
def get_preds(x,th):
  l_pred =[]
  for value in x:
    if value<=th:
      l_pred.append(1)
    else:
      l_pred.append(0)
  pred = np.array(l_pred)
  return pred

# function that takes as input the confusion matrix obtained with the method confusion_matrix() from sklearn.metrics and print it
def print_cm(cm,th):
  TN, FP, FN, TP = cm.ravel()
  plt.figure(figsize=(3, 2))
  sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', cbar=False, annot_kws={"size": 7},
              xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
  plt.xlabel('Predicted')
  plt.ylabel('Real')
  title = 'Confusion matrix with th='+str(th)
  plt.title(title+'\nTP={}, TN={}, FP={}, FN={}'.format(TP, TN, FP, FN))
  plt.show()
  return TN, FP, FN, TP

# funtcion that takes as input the TPR and FPR lists and print the ROC curve
def print_ROC_curve(FPR_list, TPR_list):
  plt.plot(FPR_list, TPR_list, linestyle='-', color='r', linewidth=3)
  plt.plot([0, 1], [0, 1], linestyle='--', color='red')
  max_TPR = max(TPR_list)
  max_TPR_index = TPR_list.index(max_TPR)
  plt.axhline(y=max_TPR, color='b', linestyle='--', label=f'Maximum TPR: {max_TPR:.2f}')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC Curve')
  plt.xlim([0, 1])
  plt.ylim([0, 1.05])
  plt.grid(True)
  plt.legend()
  plt.show()
  roc_auc = auc(FPR_list, TPR_list)
  print('AUC =', roc_auc)



## 1.   Optimization



**Set loading**

In [ ]:
subset1 = files.upload()
# upload opt1_seq.txt for the multiple sequence alignment model
# upload opt1_str.txt for the multiple structure alignment model

In [ ]:
subset2 = files.upload()
# upload opt2_seq.txt for the multiple sequence alignment model
# upload opt2_str.txt for the multiple structure alignment model

In [ ]:
test_set = files.upload()
# upload test_seq.txt for the multiple sequence alignment model
# upload test_str.txt for the multiple structure alignment model



### 1.1.   Subset1 as optimization set and subset2 as valdation set




**Dataframe creation**

In [ ]:
for file_name, content in subset1.items():
  opt1_df = pd.read_csv(file_name, sep='\t', header=None, names=['ID', 'E-value', 'Label'])

In [ ]:
opt1_df.head()

In [ ]:
opt1_df.shape

In [ ]:
for file_name, content in subset2.items():
  val1_df = pd.read_csv(file_name, sep='\t', header=None, names=['ID', 'E-value', 'Label'])

In [ ]:
val1_df.head()

In [ ]:
val1_df.shape

**Creation of two optimization arrays containing the E-values and the labels**

In [ ]:
X_opt1= opt1_df["E-value"].to_numpy()
y_opt1= opt1_df["Label"].to_numpy()

In [ ]:
X_opt1

In [ ]:
y_opt1

**Creation of two validation arrays containing the E-values and the labels**

In [ ]:
X_val1= val1_df["E-value"].to_numpy()
y_val1= val1_df["Label"].to_numpy()

In [ ]:
X_val1

In [ ]:
y_val1

**Obtaining predictions and classification report using various E-value tresholds**

In [ ]:
cols = ['Threshold','Accuracy','Pos_f1_score','Neg_f1_score','MCC','TPR','FPR']
res_opt1 = pd.DataFrame(columns=cols)
# creation of a dataframe that will be used to visualize better the results

In [ ]:
opt1_TPR_list = []
opt1_FPR_list = []

for th in [0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001, 0.0000000001, 0.00000000001]:
  y_pred1 = get_preds(X_opt1, th)
  cm = confusion_matrix(y_opt1, y_pred1)
  TN, FP, FN, TP = print_cm(cm, th)
  report = classification_report(y_opt1, y_pred1, output_dict=True)
  ac, pos_f1, neg_f1, mcc = report['accuracy'], report['1']['f1-score'], report['0']['f1-score'], matthews_corrcoef(y_opt1, y_pred1)
  print('THRESHOLD=%s\nAccuracy=%s\nPositive F1=%s\nNegative F1=%s\nMCC=%s\n'%(th,ac,pos_f1,neg_f1,mcc))
  opt1_TPR_current = TP / (TP + FN)
  opt1_FPR_current = FP / (FP + TN)
  opt1_TPR_list.append(opt1_TPR_current)
  opt1_FPR_list.append(opt1_FPR_current)
  new_res = [th, ac, pos_f1, neg_f1, mcc, opt1_TPR_current, opt1_FPR_current]
  res_opt1.loc[len(res_opt1)] = new_res


**ROC curve**

In [ ]:
print_ROC_curve(opt1_FPR_list, opt1_TPR_list)
print(res_opt1)

**Validation of the best threshold**

In [ ]:
th = # INSERT BEST THRESHOLD SELECTED
# best threshold selected for the multiple sequence alignment model is 0.000000001
# best threshold selected for the multiple structure alignment model is 0.00000001

In [ ]:
y_pred1 = get_preds(X_val1, th)
cm = confusion_matrix(y_val1, y_pred1)
TN, FP, FN, TP = print_cm(cm,th)
report = classification_report(y_val1, y_pred1, output_dict=True)
ac, pos_f1, neg_f1, mcc = report['accuracy'], report['1']['f1-score'], report['0']['f1-score'], matthews_corrcoef(y_val1, y_pred1)
print('THRESHOLD=%s\nAccuracy=%s\nPositive F1=%s\nNegative F1=%s\nMCC=%s\n'%(th,ac,pos_f1,neg_f1,mcc))



### 1.1.   Subset2 as optimization set and subset1 as valdation set




**Dataframe creation**

In [ ]:
for file_name, content in subset2.items():
  opt2_df = pd.read_csv(file_name, sep='\t', header=None, names=['ID', 'E-value', 'Label'])

In [ ]:
opt2_df.head()

In [ ]:
opt2_df.shape

In [ ]:
for file_name, content in subset1.items():
  val2_df = pd.read_csv(file_name, sep='\t', header=None, names=['ID', 'E-value', 'Label'])

In [ ]:
val2_df.head()

In [ ]:
val2_df.shape

**Creation of two optimization arrays containing the E-values and the labels**

In [ ]:
X_opt2 = opt2_df["E-value"].to_numpy()
y_opt2 = opt2_df["Label"].to_numpy()

In [ ]:
X_opt2

In [ ]:
y_opt2

**Creation of two validation arrays containing the E-values and the labels**

In [ ]:
X_val2 = val2_df["E-value"].to_numpy()
y_val2 = val2_df["Label"].to_numpy()

In [ ]:
X_val2

In [ ]:
y_val2

**Obtaining predictions and classification report using various E-value tresholds**

In [ ]:
cols = ['Threshold','Accuracy','Pos_f1_score','Neg_f1_score','MCC','TPR','FPR']
res_opt2 = pd.DataFrame(columns=cols)
# creation of a dataframe that will be used to visualize better the results

In [ ]:
opt2_TPR_list = []
opt2_FPR_list = []

for th in [0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001, 0.0000000001, 0.00000000001]:
  y_pred2 = get_preds(X_opt2, th)
  cm = confusion_matrix(y_opt2, y_pred2)
  TN, FP, FN, TP = print_cm(cm, th)
  report = classification_report(y_opt2, y_pred2, output_dict=True)
  ac, pos_f1, neg_f1, mcc = report['accuracy'], report['1']['f1-score'], report['0']['f1-score'], matthews_corrcoef(y_opt2, y_pred2)
  print('THRESHOLD=%s\nAccuracy=%s\nPositive F1=%s\nNegative F1=%s\nMCC=%s\n'%(th,ac,pos_f1,neg_f1,mcc))
  opt2_TPR_current = TP / (TP + FN)
  opt2_FPR_current = FP / (FP + TN)
  opt2_TPR_list.append(opt2_TPR_current)
  opt2_FPR_list.append(opt2_FPR_current)
  new_res = [th, ac, pos_f1, neg_f1, mcc, opt2_TPR_current, opt2_FPR_current]
  res_opt2.loc[len(res_opt2)] = new_res


**ROC curve**

In [ ]:
print_ROC_curve(opt2_FPR_list, opt2_TPR_list)
print(res_opt2)

**Validation of the best threshold**

In [ ]:
th2 = # INSERT BEST THRESHOLD SELECTED
# best threshold selected for the multiple sequence alignment model is 0.000001
# best threshold selected for the multiple structure alignment model is 0.000001

In [ ]:
y_pred2 = get_preds(X_val2, th2)
cm = confusion_matrix(y_val2, y_pred2)
TN, FP, FN, TP = print_cm(cm, th2)
report = classification_report(y_val2, y_pred2, output_dict=True)
ac, pos_f1, neg_f1, mcc = report['accuracy'], report['1']['f1-score'], report['0']['f1-score'], matthews_corrcoef(y_val2, y_pred2)
print('THRESHOLD=%s\nAccuracy=%s\nPositive F1=%s\nNegative F1=%s\nMCC=%s\n'%(th2,ac,pos_f1,neg_f1,mcc))


## 2.   Final test



**Dataframe creation**

In [ ]:
for file_name, content in test_set.items():
  test_df = pd.read_csv(file_name, sep='\t', header=None, names=['ID_proteina', 'E-value', 'Label'])

In [ ]:
test_df.head()

In [ ]:
test_df.shape

**Creation of two arrays containing the E-values and the labels**

In [ ]:
X_test = test_df['E-value'].to_numpy()
y_test = test_df['Label'].to_numpy()

In [ ]:
th = # insert the average of the best thresholds
# threshold selected for the multiple sequence alignment model is 0.00000003162
# threshold selected for the multiple structure alignment model is 0.0000001

In [ ]:
y_pred3 = get_preds(X_test, th)
cm = confusion_matrix(y_test, y_pred3)
TN, FP, FN, TP = print_cm(cm, th)
report = classification_report(y_test, y_pred3, output_dict=True)
ac, pos_f1, neg_f1, mcc = report['accuracy'], report['1']['f1-score'], report['0']['f1-score'], matthews_corrcoef(y_test, y_pred3)
print('THRESHOLD=%s\nAccuracy=%s\nPositive F1=%s\nNegative F1=%s\nMCC=%s\n'%(th,ac,pos_f1,neg_f1,mcc))